In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/LowRankVortex.jl/")

  Activating environment at `/media/mat/HDD/LowRankVortex.jl/Project.toml`


In [2]:
using Revise
using LowRankVortex
using LinearAlgebra
using Statistics
using PotentialFlow
import PotentialFlow.Plates: Plate, Points, Blobs
import PotentialFlow.Motions: reset_velocity!
import PotentialFlow.Elements
import PotentialFlow.Properties: @property
using Interpolations
using Distributions
using Test

In [16]:
@testset "Test symmetric_analytical_jacobian_position for regularized vortices with real freestream" begin

    atol = 1000*eps()

    Nv = 10
    Nx = 3*Nv

    x = rand(Nx)
    x0 = deepcopy(x)

    xsensors = collect(-2.0:0.5:10)
    sensors = complex(xsensors)
    Ny = length(sensors)
#     U = 0.0 + 0.0*im
    U = randn() + 0.0*im

    #     U = randn(ComplexF64)
    freestream = Freestream(U);

    config = let Nv = Nv,
             U = U,
             ss = sensors, Δt = 1e-3, δ = 5e-2,
             ϵX = 1e-3, ϵΓ = 1e-4,
             β = 1.0,
             ϵY = 1e-16
            VortexConfig(Nv, U, ss, Δt, δ, ϵX, ϵΓ, β, ϵY)
    end

    sys = state_to_lagrange(x, config; isblob = true)
    @test typeof(sys[1][1])<:Vortex.Blob{Float64, Float64}

    sys = vcat(sys...)

    dpdz, dpdzstar = analytical_jacobian_position(sensors, sys, freestream, 0.0)

    dpdzsym, dpdzstarsym = symmetric_analytical_jacobian_position(xsensors, sys, freestream, 0.0)

    @test isapprox(dpdz, dpdzsym, atol = atol)
    @test isapprox(dpdzstar, dpdzstarsym, atol = atol)
end

Test symmetric_analytical_jacobian_position for regularized vortices with real freestream: Test Failed at In[16]:38
  Expression: isapprox(dpdz, dpdzsym, atol = atol)
   Evaluated: isapprox(ComplexF64[-0.010114227176445662 + 0.0036600722560452383im -0.0015707431261411898 - 0.00869320600460579im … -0.0019372313971974583 - 0.006135482094892429im -0.00389124899678262 - 0.000985698553702935im; -0.015099730411931316 + 0.005785262191886782im -0.0025116896439446026 - 0.012156332426192249im … -0.005672865462103006 - 0.01587907706507375im -0.006312135704207519 - 0.001149092935343841im; … ; -0.00031092406382386727 + 0.0005666861769334492im 1.779035834662709e-5 - 0.0010232008116829174im … 0.00044443632741536504 + 0.0003616578817307229im -6.02059202024176e-5 - 0.0002598795495642104im; -0.00028874938319507705 + 0.000501943527206079im 1.360038998829912e-5 - 0.0009151644840659223im … 0.0003959379532260728 + 0.0003218384326424797im -5.779897244337514e-5 - 0.00023068254567507014im], ComplexF64[-0.00322

LoadError: [91mSome tests did not pass: 1 passed, 2 failed, 0 errored, 0 broken.[39m